In [1]:
import torch
from resnetModel import *
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from pytorch_lightning import Trainer, seed_everything
seed_everything(77)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model=torch.load('es9-dl-model2.pt')
model=model.to(device)

In [2]:
transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

trainset = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=transform_train)
valset = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=transform_test)
testset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
        testset, batch_size=100, shuffle=False, num_workers=2)
criterion = nn.CrossEntropyLoss()

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
from utils import progress_bar
def test(net,criterion):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    all_predicts=[]
    all_targets=[]
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                        % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

In [4]:
test(model,criterion)

 [================================================================>]  Step: 93ms | Tot: 3s136ms | Loss: 0.419 | Acc: 85.780% (8578/1000 100/100 
